# Analyze data from NewsDataAPIClient

# 1. Imports

## 1.1 Packages

In [65]:
import pandas as pd
import sys
import yaml

## 1.2 Options

In [66]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
path_credentials = '../conf/local/credentials.yml'

In [68]:
with open(path_credentials) as file:
    credentials = yaml.safe_load(file)

## 1.3 Functions

In [69]:
sys.path.append('../src/positive_news/pipelines/')

from data_engineering.nodes import (  # noqa: E402
    drop_unavailable_columns,
    drop_unnecessary_columns,
    login_to_feature_store,
    rename_columns,
    request_news,
    send_data_to_feature_store,
    set_date_format,
    set_string_columns,
)


# 2. Data

In [70]:
# df_news = request_news(api_key=credentials['news_data_api_key'], max_result=2)
# df_news.to_parquet('../data/01_raw/df_news_test.parquet')

In [71]:
df_news = pd.read_parquet('../data/01_raw/df_news_test.parquet')

In [72]:
df_news = drop_unavailable_columns(df_news)
df_news = drop_unnecessary_columns(df_news, cols_to_remove=["video_url", "image_url", "source_priority", "source_icon"])
df_news = set_string_columns(df_news, cat_columns=["country", "category", "creator", "source_url"])
df_news = set_date_format(df_news)
df_news = rename_columns(df_news)

In [73]:
df_news.sample(2)

,article_id,title,link,keywords,creator,description,publication_date,source_id,source_name,source_url,language,country,category,duplicate
25,34b39e322f13f8a88918542c303a5913,Dramatic vid shows panicking Russians blow up ...,https://www.thesun.ie/news/13620341/kursk-inva...,"[ukraine war, news, global politics]",['James Halpin'],None,2024-08-15 08:41:13,thesun_ie,The Irish Sun,https://www.thesun.ie,english,['ireland'],['top'],True
8,336a50fb35d843bd790cf0d4099ba430,Prediction: These Could Be the Best-Performing...,https://www.fool.com/investing/2024/08/15/pred...,"[d07a79b1-e897-4b1b-9408-ff9eee38884f, investi...",['newsfeedback@fool.com (Daniel Miller)'],Finding the right EV stocks to invest in for t...,2024-08-15 08:46:00,fool,Fool,https://www.fool.com,english,['united states of america'],['business'],False


In [74]:
df_news.columns

Index(['article_id', 'title', 'link', 'keywords', 'creator', 'description',
       'publication_date', 'source_id', 'source_name', 'source_url',
       'language', 'country', 'category', 'duplicate'],
      dtype='object')

In [75]:
# login_to_feature_store(
#     host=credentials['hopsworks']['host'],
#     port=credentials['hopsworks']['port'],
#     api_key_value=credentials['hopsworks']['api_key_value']
# )

import hopsworks

project = hopsworks.login()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/24732


In [76]:
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [79]:
df_fg = fs.get_or_create_feature_group(
    name="positive_news",
    version=2,
    description="News in English",
    online_enabled=True,
    primary_key=['article_id'],
)

In [80]:
df_fg.insert(df_news, write_options={"wait_for_job": False})

RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/24732/featurestores/24652/featuregroups). Server response: 
HTTP code: 500, HTTP reason: Internal Server Error, body: b'{"errorCode":270063,"usrMsg":"Problem getting secrets for the JDBC connection to the online FS","errorMsg":"Could not get JDBC connection for the online featurestore"}', error code: 270063, error msg: Could not get JDBC connection for the online featurestore, user msg: Problem getting secrets for the JDBC connection to the online FS